<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/colorCorrection/tests/CURL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CURL: Neural Curve Layers for Global Image Enhancement

[GitHub](https://github.com/sjmoran/CURL)

## 1. Preparations

Before start, make sure that you choose

Runtime Type = Python 3
Hardware Accelerator = GPU

In [ ]:
!nvidia-smi

## 2. linking nextcloud

Connecting to the external NextCloud drive 

In [ ]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

input_folder, output_folder = create_io(database=nextcloud,topic='bgRemoval',library='CURL')

## 3. clone GIT repo

In [ ]:
import os
root_path = '/content/CURL'

# clone the repository
if not os.path.exists(root_path):
  !git clone https://github.com/sjmoran/CURL {root_path}

%ls

## 4. Setting up the environment

In [ ]:
# installing CURL
%cd {root_path}
!pip install -q -r requirements.txt